## Imports

In [1]:
import random                  
from fractions import Fraction
import numpy as np
import pandas as pd
from music21 import chord, corpus, environment, meter, midi, note, stream, tempo, converter

In [2]:
def play(score):
    #Shortcut to play a stream
    midi.realtime.StreamPlayer(score).play()

## Setup

In [225]:
score = converter.parse('score.mxl')
r_hand = score.parts[0]
l_hand = score.parts[1]
r_measures = r_hand.getElementsByClass('Measure')
l_measures = l_hand.getElementsByClass('Measure')

piece_end = r_measures[-1].offset + 4

## Pitch degeneration

In [226]:
#def shift_notes_pitch(delta_fun, notes):
#    for n in notes:
#        cents = n.pitch.microtone.cents
#        n.pitch.microtone = cents + delta_fun(n.offset)
#        print(n.offset)

def shift_pitch_fun(delta_fun, start_offset=0, end_offset=piece_end, include_end=False):
    notes = score.flat.getElementsByOffset(start_offset, end_offset, includeEndBoundary=include_end, classList=[note.Note])
    for n in notes:
        cents = n.pitch.microtone.cents
        n.pitch.microtone = cents + delta_fun(n.offset)
    
    chords = score.flat.getElementsByOffset(start_offset, end_offset, includeEndBoundary=include_end, classList=[chord.Chord])
    for c in chords:
        for n in c.notes:
            cents = n.pitch.microtone.cents
            n.pitch.microtone = cents + delta_fun(c.offset)

def shift_pitch(delta, start_offset=0, end_offset=piece_end, include_end=False):
    shift_pitch_fun(lambda x: delta, start_offset, end_offset, include_end)

In [229]:
shift_pitch(25, 4*11)
shift_pitch(50, 4*11*2)
shift_pitch(-75, 4*11*3)

def gradual_shift(offset):
    progression = offset/piece_end
    max_delta = 100*progression
    return random.uniform(-max_delta, max_delta)
    
shift_pitch_fun(gradual_shift)

## Export to MIDI

In [231]:
score.write('midi', fp='out.mid')

'out.mid'